In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-09-09 18:34:09 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-09-09 18:34:09 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-09-09 18:34:09 __main__ INFO     transformers.__version__='4.55.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.1-70B-Instruct"
# model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-09-09 18:34:12 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-09 18:34:12 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-09 18:34:12 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-09-09 18:34:12 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-09-09 18:34:13 src.models WARNING  google/gemma-2-27b-it not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-09-09 18:34:13 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-09-09 18:34:13 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/config.json HTTP/1.1" 200 0
2025-09-09 18:34:13 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-09-09 18:34:13 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/google/gemma-2-27b-it/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

2025-09-09 18:34:33 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-09-09 18:34:34 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-09-09 18:34:34 src.models INFO     loaded model <google/gemma-2-27b-it> | size: 51931.626 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.selection.data import SelectOneTask, CountingTask, YesNoTask, SelectFirstTask
from src.selection.data import SelectionSample, CountingSample, YesNoSample

#################################################################################
# TASK_CLS = SelectOneTask
TASK_CLS = CountingTask
prompt_template_idx = 0
N_DISTRACTORS = 5
OPTION_STYLE = "single_line"
#################################################################################

select_task = TASK_CLS.load(
    path=os.path.join(
        env_utils.DEFAULT_DATA_DIR, 
        "selection", 
        # "profession.json"
        # "nationality.json"
        "objects.json"
    )
)

print(select_task)

CountingTask: (different objects)
Categories: fruit(15), vehicle(15), furniture(15), animal(15), music instrument(15), clothing(15), electronics(15), sport equipment(15), kitchen appliance(15), vegetable(14), building(15), office supply(15), bathroom item(15), flower(15), tree(15), jewelry(15)



In [23]:
from src.functional import predict_next_token, generate_with_patch

sample = select_task.get_random_sample(
    mt=mt, 
    category="vehicle",
    filter_by_lm_prediction=True,
    prompt_template_idx=1,
    n_count=4,
    n_distractors=3,
)
print(f'"{sample.prompt()}" >> {sample.count}')

gen = generate_with_patch(
    mt=mt, 
    inputs=sample.prompt(),
    n_gen_per_prompt=1,
    remove_prefix=True,
    max_new_tokens=10,
    do_sample=False,
)
print(gen) # expect sample.count

"Items: Airplane, Necklace, Jacket, Boxing gloves, Tractor, Ambulance, Truck
How many vehicles are in this list?
Answer:" >> 4
[' 4\n\nExplanation:\n\nThe vehicles in the']


In [24]:
sample.prediction

OrderedDict([(11016,
              (12,
               PredictedToken(token=' Four', prob=5.3882598876953125e-05, logit=13.0625, token_id=11016, metadata=None))),
             (18457,
              (18,
               PredictedToken(token=' Five', prob=1.7523765563964844e-05, logit=11.9375, token_id=18457, metadata=None))),
             (11176,
              (52,
               PredictedToken(token=' Three', prob=1.6242265701293945e-06, logit=9.5625, token_id=11176, metadata=None))),
             (16544,
              (74,
               PredictedToken(token=' Six', prob=7.674098014831543e-07, logit=8.8125, token_id=16544, metadata=None))),
             (7020,
              (110,
               PredictedToken(token=' Two', prob=3.110617399215698e-07, logit=7.90625, token_id=7020, metadata=None))),
             (3428,
              (182,
               PredictedToken(token=' One', prob=1.1781230568885803e-07, logit=6.9375, token_id=3428, metadata=None))),
             (21117,
          

In [9]:
from src.selection.data import get_counterfactual_samples_interface

counterfactual_sampler = get_counterfactual_samples_interface[select_task.task_name]

patch_sample, clean_sample = counterfactual_sampler(
    mt=mt,
    task=select_task,
    patch_category="fruit",
    clean_category="vehicle",
    filter_by_lm_prediction=False,
    prompt_template_idx=1,
    option_style=OPTION_STYLE,
    distinct_options=True,
    n_options=5
)

# patch_sample.default_option_style = "single_line"
# clean_sample.default_option_style = "numbered"

print("-" * 100)
print(patch_sample.prompt(), ">>", patch_sample.count)
print(clean_sample.prompt(), ">>", clean_sample.count)

clean_options=['Mango', 'Watermelon', 'Orange', 'Car', 'Grape']
patch_options=['Kiwi', 'Banana', 'Peach', 'Pear', 'Tractor']
clean_category='vehicle'
patch_category='fruit'
clean_options=['Mango', 'Watermelon', 'Orange', 'Car', 'Grape']
patch_options=['Kiwi', 'Banana', 'Peach', 'Pear', 'Tractor']
----------------------------------------------------------------------------------------------------
Items: Kiwi, Banana, Peach, Pear, Tractor
How many fruits are in this list?
Answer: >> 4
Items: Mango, Watermelon, Orange, Car, Grape
How many vehicles are in this list?
Answer: >> 1
